In [10]:
!pip install streamlit

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   --------- ------------------------------ 2.4/9.8 MB 11.8 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.8 MB 11.8 MB/s eta 0:00:01
   --------------------------- ------------ 6.8/9.8 MB 11.2 MB/s eta 0:00:01
   -------------------------------------- - 9.4/9.8 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.2 MB 11.9 MB/s eta 0:00:02
   ------- -------------------------------- 5.0/25.2 MB 11.8 MB/s eta 0:00:02
   ------------ --------------------------- 7.6/25.2 MB 11.8 MB/s eta 0:00:02
   --------------- ------------------------ 10.0/25.2 MB 11.8 MB/s eta 0:00:02
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from fastapi import FastAPI
from pydantic import BaseModel
from datetime import datetime
from typing import Literal
import math
import uuid
import sqlite3
import streamlit as st
import requests
import json

app = FastAPI()

# --- Banco de dados SQLite ---
conn = sqlite3.connect("usuarios.db", check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS usuarios (
    id TEXT PRIMARY KEY,
    timestamp TEXT,
    despesa_mensal REAL,
    tipo_trabalho TEXT,
    n_dependentes INTEGER,
    valor_atual REAL,
    meta REAL,
    prazo_anos REAL,
    aporte_atual REAL,
    taxa_retorno_anual REAL,
    reserva_ideal REAL,
    aporte_necessario REAL,
    tempo_estimado REAL
)
''')
conn.commit()

# --- Modelo de entrada ---
class UsuarioInput(BaseModel):
    despesa_mensal: float
    tipo_trabalho: Literal['CLT', 'Autônomo']
    n_dependentes: int
    valor_atual: float
    meta: float
    prazo_anos: float
    aporte_atual: float
    taxa_retorno_anual: float

# --- Funções de cálculo ---
def calcular_reserva(despesa_mensal, tipo_trabalho, n_dependentes):
    meses_reserva = 6 if tipo_trabalho.lower() == 'clt' else 12
    reserva = (despesa_mensal * meses_reserva) + (n_dependentes * 1000)
    return reserva

def calcular_aporte_necessario(meta, prazo_anos, taxa_retorno_anual):
    i = taxa_retorno_anual / 12 / 100
    n = prazo_anos * 12
    if n == 0:
        return 0
    if i == 0:
        return meta / n
    aporte = meta * i / ((1 + i) ** n - 1)
    return aporte

def calcular_tempo_para_meta(valor_atual, aporte_mensal, taxa_retorno_anual, meta):
    i = taxa_retorno_anual / 12 / 100
    saldo = valor_atual
    meses = 0
    while saldo < meta and meses < 1000:
        saldo = saldo * (1 + i) + aporte_mensal
        meses += 1
    anos = meses / 12
    return round(anos, 1)

# --- Endpoint principal ---
@app.post("/simular")
def simular_dados(dados: UsuarioInput):
    reserva_ideal = calcular_reserva(dados.despesa_mensal, dados.tipo_trabalho, dados.n_dependentes)
    aporte_necessario = calcular_aporte_necessario(dados.meta, dados.prazo_anos, dados.taxa_retorno_anual)
    tempo_estimado = calcular_tempo_para_meta(dados.valor_atual, dados.aporte_atual, dados.taxa_retorno_anual, dados.meta)

    id_user = str(uuid.uuid4())
    timestamp = datetime.now().isoformat()

    cursor.execute("""
        INSERT INTO usuarios VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        id_user, timestamp,
        dados.despesa_mensal, dados.tipo_trabalho, dados.n_dependentes,
        dados.valor_atual, dados.meta, dados.prazo_anos,
        dados.aporte_atual, dados.taxa_retorno_anual,
        reserva_ideal, aporte_necessario, tempo_estimado
    ))
    conn.commit()

    return {
        "reserva_ideal": round(reserva_ideal, 2),
        "aporte_necessario": round(aporte_necessario, 2),
        "tempo_estimado_para_meta_anos": tempo_estimado
    }

# --- Interface Streamlit ---
st.title("Finanças com Dados 💰")

with st.form("simulador_form"):
    despesa_mensal = st.number_input("Despesa Mensal", min_value=0.0)
    tipo_trabalho = st.selectbox("Tipo de Trabalho", ["CLT", "Autônomo"])
    n_dependentes = st.number_input("Número de Dependentes", min_value=0, step=1)
    valor_atual = st.number_input("Valor Atual Investido", min_value=0.0)
    meta = st.number_input("Meta Financeira (R$)", min_value=0.0)
    prazo_anos = st.number_input("Prazo (anos)", min_value=0.0)
    aporte_atual = st.number_input("Aporte Mensal Atual", min_value=0.0)
    taxa_retorno_anual = st.number_input("Taxa de Retorno Anual (%)", min_value=0.0)
    enviar = st.form_submit_button("Simular")

if enviar:
    dados = {
        "despesa_mensal": despesa_mensal,
        "tipo_trabalho": tipo_trabalho,
        "n_dependentes": n_dependentes,
        "valor_atual": valor_atual,
        "meta": meta,
        "prazo_anos": prazo_anos,
        "aporte_atual": aporte_atual,
        "taxa_retorno_anual": taxa_retorno_anual,
    }
    response = requests.post("http://localhost:8000/simular", data=json.dumps(dados))
    if response.status_code == 200:
        resultado = response.json()
        st.success("Simulação realizada com sucesso!")
        st.metric("Reserva de Emergência Ideal (R$)", f"{resultado['reserva_ideal']:.2f}")
        st.metric("Aporte Mensal Necessário (R$)", f"{resultado['aporte_necessario']:.2f}")
        st.metric("Tempo Estimado para Meta (anos)", f"{resultado['tempo_estimado_para_meta_anos']:.1f}")
    else:
        st.error("Erro ao processar a simulação. Verifique os dados e tente novamente.")


ModuleNotFoundError: No module named 'streamlit'